In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

from cbsim.net import Net
from cbsim.request import Request
from cbsim.stochastic import Stochastic
from cbsim.experiment import read_exper_data
from cbsim.experiment import equal_var_test
from cbsim.experiment import pooled_var_t_test

In [9]:
net = Net()
net.load_from_file(fnodes='ss-nodes.csv', flinks='ss-links.csv')

days = 6 # 7:00am-11:30am
fdata = {
    0: [[114, 108, 107, 51, 139, 114], [15, 11, 7, 5, 8, 2]],   # Calle San Juan
    1: [[103, 102, 88, 35, 118, 101], [17, 7, 6, 13, 10, 9]],   # Calle Inigo
    10: [[130, 127, 106, 47, 117, 78], [3, 11, 7, 5, 14, 18]],  # Boulevard
    21: [[47, 23, 43, 21, 45, 35], [25, 33, 27, 19, 42, 19]]    # Calle Ijentea
}
loadpoints = {0: 'A', 21: 'B'}
outlets = [3, 6, 19]

flows = {}
for f in fdata:
    d = fdata[f]
    flows[f] = int(np.array([d[0][i] + d[1][i] for i in range(days)]).min()) # MEAN
_total = sum(flows.values()) # total number of requests to generate

probs = { 'R': 1, 'H': 1, 'S': 1, 'P': 1, 'W': 1 }
# probs = { 'R': 0.2, 'H': 0.05, 'S': 0.2, 'P': 0.15, 'W': 0.05 }

entries = sorted(flows.keys())

net.auto_regions(entries)
net.set_regions()
zones = sorted([r.code for r in net.regions])

itscs = [nd for nd in net.nodes if nd.type == 'N']
clients = [nd for nd in net.nodes if nd.type != 'N' and nd.type != 'L']

print(flows, _total)


def simulate(weight=0.15, capacity=0.15):

    # 1) generate requests
    generated = 0
    reqs = []
    while generated < _total:
        dst = np.random.choice(clients)
        if np.random.random() < probs[dst.type]:
            req = Request(0, None, dst)
            generated += 1
            reqs.append(req)
    # 2) generate demand using the proposed aproach
    reqs = net.gen_demand(flows=flows,
                        probs=probs,
                        requests=reqs,
                        s_weight=Stochastic(law=1, location=weight, scale=0.3*weight),
                        traditional=False)
    # 3) calculate distances savings
    dds = net.simulate(requests=reqs, outlets=outlets, loadpoints=None, capacity=capacity)
    bds = net.simulate(requests=reqs, outlets=outlets, loadpoints=loadpoints.keys(), capacity=capacity)
    # print(len(dds[0]), len(dds[1]), sum(dds[0]), sum(dds[1]))
    # print(len(bds[0]), len(bds[1]), sum(bds[0]), sum(bds[1]), sum(bds[0]) + sum(bds[1]))
    #print(sum(dds[0]) - sum(bds[0]) - sum(bds[1]))
    #return dds[0], bds[0], bds[1]
    return sum(dds[0]) - sum(bds[0]) - sum(bds[1])


{0: 56, 1: 48, 10: 52, 21: 40} 196


In [12]:
deltas = np.array([4.558, 7.947, 10.251])
fs = np.array([196, 387, 500])
kn = 4 * 4
fs / kn, deltas / (fs / kn)

(array([12.25  , 24.1875, 31.25  ]),
 array([0.37208163, 0.32855814, 0.328032  ]))

In [13]:
deltas = np.array([8.037, 9.187, 11.145])
fs = np.array([1028, 1186, 1435])
kn = 7 * 8
fs / kn, deltas / (fs / kn)

(array([18.35714286, 21.17857143, 25.625     ]),
 array([0.43781323, 0.43378752, 0.43492683]))

<H1>Run simulations

In [10]:
[simulate() for _ in range(50)]

[23.460000000000043,
 25.69199999999999,
 23.80700000000011,
 24.16500000000001,
 25.07900000000003,
 25.056000000000033,
 23.44000000000002,
 26.16300000000002,
 26.264000000000046,
 28.910000000000018,
 23.862999999999992,
 23.72399999999998,
 19.458000000000062,
 26.30400000000007,
 23.160000000000057,
 28.167000000000066,
 20.20800000000006,
 27.22700000000005,
 23.972000000000016,
 27.79100000000002,
 21.14500000000007,
 22.975000000000065,
 23.692000000000004,
 26.00200000000008,
 24.335000000000015,
 25.518000000000033,
 23.76200000000004,
 23.15600000000002,
 23.20400000000009,
 22.997000000000025,
 25.179000000000002,
 23.79900000000003,
 24.059000000000005,
 26.3100000000001,
 19.434999999999967,
 25.211,
 23.112000000000016,
 24.727000000000032,
 22.18200000000004,
 24.265000000000075,
 26.247000000000064,
 20.340000000000003,
 25.539999999999964,
 27.925999999999988,
 26.208999999999993,
 28.957000000000054,
 28.08199999999999,
 30.596999999999962,
 23.564,
 26.565000000000

In [7]:
ds1 = [53.750999999999955,
 41.628999999999806,
 40.23799999999975,
 43.66999999999978,
 47.54099999999969,
 42.16299999999957,
 51.87499999999975,
 48.78699999999975,
 43.85299999999982,
 47.50099999999991,
 47.820999999999756,
 42.095999999999755,
 39.895999999999944,
 41.51099999999978,
 43.40099999999973,
 44.80299999999981,
 41.36099999999997,
 44.03799999999998,
 43.730999999999796,
 46.16599999999967,
 52.785999999999774,
 48.233999999999654,
 48.454999999999885,
 46.794000000000025,
 46.017999999999795,
 48.81299999999985,
 45.22399999999965,
 44.77799999999982,
 44.66299999999981,
 47.71999999999966,
 51.42099999999979,
 45.42699999999972,
 40.88599999999979,
 43.29399999999977,
 47.57499999999992,
 48.14599999999967,
 47.543999999999855,
 42.00999999999993,
 49.53999999999963,
 48.56699999999989,
 45.13999999999999,
 44.47499999999972,
 41.39299999999991,
 43.32399999999998,
 43.84599999999971,
 50.93,
 45.98199999999986,
 46.68399999999983,
 52.57799999999978,
 44.37799999999998]

In [8]:
ds0 = [43.92199999999985,
 42.860999999999805,
 52.34099999999984,
 47.268999999999735,
 48.46599999999986,
 43.44899999999966,
 51.668999999999656,
 46.29799999999983,
 50.01099999999973,
 48.91699999999977,
 43.91699999999983,
 46.06099999999985,
 48.86999999999979,
 45.16299999999978,
 39.76999999999985,
 41.24199999999986,
 46.52599999999962,
 51.987999999999786,
 46.160999999999845,
 42.56899999999991,
 44.305999999999614,
 48.15399999999974,
 39.07700000000001,
 50.48799999999985,
 45.91499999999981,
 48.57799999999976,
 50.676000000000045,
 41.57799999999991,
 44.23899999999995,
 45.21799999999977,
 48.40499999999987,
 47.31899999999968,
 40.749999999999744,
 49.299999999999656,
 44.00999999999965,
 53.07399999999999,
 44.29599999999986,
 48.49399999999992,
 47.463999999999935,
 43.54899999999969,
 52.4509999999998,
 45.22299999999985,
 42.357999999999755,
 46.35599999999974,
 50.04200000000002,
 38.127999999999766,
 46.40399999999985,
 47.252999999999815,
 46.860999999999805,
 41.97599999999994]

In [9]:
equal_var_test(ds0, ds1)

{'smaller': '2nd argument',
 'df1': 49,
 'df2': 49,
 'var1': 12.869849182400012,
 'var2': 11.690756120399968,
 'Fe': 1.10085686929544,
 'Ft': 1.7621885352431106,
 'result': True}

In [10]:
pooled_var_t_test(ds0, ds1)

{'xmean': 46.188239999999816,
 'ymean': 45.84913999999983,
 'xvar': 12.869849182400012,
 'yvar': 11.690756120399968,
 'pvar': 12.28030265139999,
 'df': 98,
 't': 0.48383051869998484,
 'talpha': 1.984467454426692,
 'result': True}

In [11]:
# Min flows
ds1 = [23.460000000000043,
 25.69199999999999,
 23.80700000000011,
 24.16500000000001,
 25.07900000000003,
 25.056000000000033,
 23.44000000000002,
 26.16300000000002,
 26.264000000000046,
 28.910000000000018,
 23.862999999999992,
 23.72399999999998,
 19.458000000000062,
 26.30400000000007,
 23.160000000000057,
 28.167000000000066,
 20.20800000000006,
 27.22700000000005,
 23.972000000000016,
 27.79100000000002,
 21.14500000000007,
 22.975000000000065,
 23.692000000000004,
 26.00200000000008,
 24.335000000000015,
 25.518000000000033,
 23.76200000000004,
 23.15600000000002,
 23.20400000000009,
 22.997000000000025,
 25.179000000000002,
 23.79900000000003,
 24.059000000000005,
 26.3100000000001,
 19.434999999999967,
 25.211,
 23.112000000000016,
 24.727000000000032,
 22.18200000000004,
 24.265000000000075,
 26.247000000000064,
 20.340000000000003,
 25.539999999999964,
 27.925999999999988,
 26.208999999999993,
 28.957000000000054,
 28.08199999999999,
 30.596999999999962,
 23.564,
 26.56500000000004]
ds0 = [23.088000000000008,
 25.448999999999998,
 25.248000000000015,
 27.420000000000034,
 26.982000000000017,
 30.48000000000005,
 25.716000000000037,
 22.497000000000014,
 24.33999999999996,
 22.398000000000074,
 21.817000000000014,
 24.276999999999976,
 26.28499999999998,
 22.09200000000002,
 21.034,
 24.253000000000032,
 25.35499999999998,
 25.59500000000002,
 22.147000000000045,
 26.54200000000006,
 27.185000000000024,
 30.147000000000062,
 20.06700000000001,
 27.02700000000005,
 23.81500000000008,
 25.27199999999999,
 26.59100000000004,
 22.50100000000008,
 27.822000000000056,
 24.827000000000016,
 23.764000000000028,
 30.127999999999997,
 23.319000000000013,
 25.803999999999995,
 24.51399999999998,
 24.000000000000018,
 29.89100000000005,
 26.242000000000115,
 28.482000000000088,
 28.699000000000034,
 24.959000000000092,
 23.005999999999982,
 24.96100000000006,
 27.92700000000008,
 25.527000000000047,
 27.07400000000011,
 24.011000000000013,
 24.927000000000078,
 24.669000000000015,
 27.99500000000004]

In [12]:
equal_var_test(ds0, ds1)

{'smaller': '2nd argument',
 'df1': 49,
 'df2': 49,
 'var1': 5.762247990400048,
 'var2': 5.660417678399983,
 'Fe': 1.0179898936413556,
 'Ft': 1.7621885352431106,
 'result': True}

In [13]:
pooled_var_t_test(ds0, ds1)

{'xmean': 25.363360000000036,
 'ymean': 24.70004000000003,
 'xvar': 5.762247990400048,
 'yvar': 5.660417678399983,
 'pvar': 5.7113328344000145,
 'df': 98,
 't': 1.3877919418416915,
 'talpha': 1.984467454426692,
 'result': True}

In [14]:
# Max flows
ds1 = [56.65099999999976,
 56.96099999999975,
 62.8789999999996,
 61.090999999999894,
 57.05699999999975,
 57.47200000000001,
 64.82799999999966,
 69.44799999999978,
 63.766999999999804,
 56.25100000000006,
 59.944999999999645,
 69.4299999999997,
 64.85499999999988,
 58.0409999999999,
 55.62899999999967,
 57.9239999999998,
 55.88000000000007,
 64.46499999999986,
 55.26299999999987,
 58.266999999999854,
 59.386999999999766,
 67.77999999999977,
 56.62299999999998,
 60.51299999999952,
 63.0159999999999,
 58.072999999999965,
 63.90299999999953,
 61.39999999999978,
 53.53199999999976,
 62.07399999999961,
 62.02499999999983,
 61.93700000000004,
 60.78599999999975,
 59.17499999999986,
 58.90199999999995,
 61.8990000000002,
 67.51799999999983,
 68.2459999999997,
 64.7629999999998,
 62.10299999999974,
 59.648000000000074,
 58.44999999999994,
 62.55499999999998,
 58.180999999999855,
 59.366999999999784,
 65.2299999999998,
 66.37699999999981,
 54.938999999999766,
 50.73899999999963,
 54.906999999999734]
ds0 = [57.23299999999999,
 59.94600000000007,
 60.23799999999986,
 56.06299999999995,
 60.240999999999914,
 61.896999999999885,
 60.83300000000003,
 56.5309999999998,
 57.53199999999964,
 66.51599999999996,
 62.46599999999986,
 56.875999999999955,
 61.38099999999996,
 61.95700000000006,
 59.18499999999974,
 58.127999999999666,
 65.90099999999978,
 62.90199999999975,
 69.18699999999981,
 60.93399999999989,
 62.843999999999845,
 50.39399999999972,
 63.414999999999814,
 61.86999999999986,
 58.278000000000084,
 55.809999999999775,
 60.12499999999977,
 52.62200000000009,
 68.68900000000006,
 57.480000000000075,
 60.317999999999806,
 61.4619999999998,
 66.2129999999998,
 63.076999999999956,
 55.57699999999988,
 67.9069999999999,
 55.53699999999986,
 63.68600000000011,
 70.24099999999976,
 65.69199999999992,
 60.00200000000002,
 61.73200000000008,
 61.5809999999999,
 57.64299999999986,
 59.29999999999972,
 61.992999999999874,
 56.59599999999985,
 65.48899999999995,
 65.08599999999996,
 59.733999999999725]

In [15]:
equal_var_test(ds0, ds1)

{'smaller': '1st argument',
 'df1': 49,
 'df2': 49,
 'var1': 18.042498438399836,
 'var2': 17.163060960000063,
 'Fe': 1.0512401302104195,
 'Ft': 1.7621885352431106,
 'result': True}

In [16]:
pooled_var_t_test(ds0, ds1)

{'xmean': 60.926799999999886,
 'ymean': 60.60303999999982,
 'xvar': 17.163060960000063,
 'yvar': 18.042498438399836,
 'pvar': 17.60277969919995,
 'df': 98,
 't': 0.38583584210533345,
 'talpha': 1.984467454426692,
 'result': True}